# Public transport data from Geofabrik on IDF scale
This data is later subsetted to GPM in QGIS.
WARNING: I DID NOT RERUN THIS SINCE CHANGING EPSGs, MAKE SURE TO RUN IT AGAIN!

In [1]:
import shapely.wkb
import pandas as pd 
import geopandas as gpd
from pyrosm import OSM
from pyrosm import get_data
import matplotlib.pyplot as plt
import networkx as nx

plt.rcParams['figure.figsize'] = (15, 15)
crs_fr = 2154


/Users/auriane/opt/miniconda3/envs/gds/lib/python3.8/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# Get data on IDF level
fp = get_data("Ile_de_France")

# Initialize the OSM parser object
osm = OSM(fp)

Downloaded Protobuf data 'ile-de-france-latest.osm.pbf' (279.98 MB) to:
'/var/folders/2m/lkz691q11b72bpprh6_v34_h0000gn/T/pyrosm/ile-de-france-latest.osm.pbf'


In [7]:
import osmium
import shapely.wkb
import pandas as pd
import geopandas as gpd


def merge_two_dicts(x, y):
    z = x.copy()   # start with keys and values of x
    z.update(y)    # modifies z with keys and values of y
    return z

class AdminAreaHandler(osmium.SimpleHandler):
    def __init__(self):
        osmium.SimpleHandler.__init__(self)

        self.areas = []
        self.wkbfab = osmium.geom.WKBFactory()

    def area(self, a):            
        if  "admin_level" in a.tags:

            wkbshape = self.wkbfab.create_multipolygon(a)
            shapely_obj = shapely.wkb.loads(wkbshape, hex=True)

            area = { "id": a.id, "geo": shapely_obj }
            area = merge_two_dicts(area, a.tags)

            self.areas.append(area)
        
handler = AdminAreaHandler()

# path to file to local drive
# download from https://download.geofabrik.de/index.html
#osm_file = osm

# start data file processing
handler.apply_file(fp, locations=True, idx='flex_mem')

df = pd.DataFrame(handler.areas)

In [20]:
gdf = gpd.GeoDataFrame(df, geometry="geo")
gdf

,id,geo,admin_level,boundary,name,population,population:date,postal_code,ref:INSEE,source:population,...,ref:FR:MGP,name:gd,manufacturer,source:name:br,name:az,name:pt,name:sr-Latn,name:zh-Hant,ref:nuts:1,ref:nuts:2
0,55263,"MULTIPOLYGON (((2.12315 48.75362, 2.12323 48.7...",8,administrative,Les Loges-en-Josas,1629,2019,78350,78343,INSEE 2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,62691,"MULTIPOLYGON (((2.03032 48.82932, 2.03039 48.8...",8,administrative,Rennemoulin,110,2019,78590,78518,INSEE 2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,62563,"MULTIPOLYGON (((2.02479 48.81185, 2.02608 48.8...",8,administrative,Fontenay-le-Fleury,13466,2019,78330,78242,INSEE 2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55947,"MULTIPOLYGON (((1.99291 48.79992, 1.99402 48.7...",8,administrative,Bois-d'Arcy,15325,2019,78390,78073,INSEE 2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,68115,"MULTIPOLYGON (((2.20849 48.66619, 2.21515 48.6...",8,administrative,Nozay,4511,2019,91620,91458,INSEE 2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1910,4378773,"MULTIPOLYGON (((2.34667 48.86443, 2.34675 48.8...",10,administrative,Quartier de Bonne-Nouvelle,NaN,NaN,75002,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1911,160451,"MULTIPOLYGON (((2.59648 48.54527, 2.60099 48.5...",8,administrative,Vert-Saint-Denis,8238,2019,77240,77495,INSEE 2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1912,160271,"MULTIPOLYGON (((2.56625 48.56371, 2.56630 48.5...",8,administrative,Cesson,10833,2019,77240,77067,INSEE 2022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913,5950449,"MULTIPOLYGON (((2.44588 48.84790, 2.44601 48.8...",10,administrative,Domaine du Bois,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# for QGIS
GPM.to_file('data/raw/GPM_geometry/GPM.geojson', driver = 'GeoJSON')
gdf.to_file('data/raw/communes.gpkg')


NameError: name 'GPM' is not defined

In [ ]:
IRIS_fr = gpd.read_file('data/raw/IRIS_shapes/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2022-06-00180/CONTOURS-IRIS_2-1_SHP_LAMB93_FXX-2022/CONTOURS-IRIS.shp')


In [5]:
IRIS_fr

NameError: name 'IRIS_fr' is not defined

# Bus

In [ ]:
# Get bus stops and routes
# =====================================

custom_filter = {'public_transport':['stop_position'],'route':['bus']}
bus_IDF = osm.get_pois(custom_filter=custom_filter)
bus_IDF = bus_stops_IDF.to_crs(crs_fr)

bus_IDF["bus_stop"] = bus_stops_IDF["public_transport"]
bus_IDF["bus_route"] = bus_stops_IDF["route"]

IOStream.flush timed out


In [ ]:
# Plot for test 
ax = plt.axes()

bus_IDF.plot(column='bus_route', ax=ax, linewidth = 0.5, edgecolor = 'red')
bus_IDF.plot(column='bus_stop', ax=ax, markersize=1.5)

In [ ]:
bus_IDF.head()

In [ ]:
# Save to geojson to use in QGIS 
#bus_IDF = pois
bus_IDF.to_file('data/processed/bus_IDF.geojson', driver='GeoJSON')

# Metro

In [6]:
# Get metro stops and routes
# =====================================

# TODO find out correct keywords
custom_filter = {'public_transport':['stop_position'],'route':['metro']}
metro_stops_IDF = osm.get_pois(custom_filter=custom_filter)

/Users/auriane/opt/miniconda3/envs/gds/lib/python3.8/site-packages/pyrosm/pyrosm.py:541: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  gdf = get_poi_data(self._nodes,
